In [47]:
import numpy as np
import pandas as pd
import math
import os
from typing import Tuple, List

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [48]:
#Reading file

def parse_xy_data(filepath, delimiter=None):
    """
    Reads a CSV-like file with x, y data and returns a list of (x, y) tuples.

    This function is designed to handle different delimiters (like ';' or ' ')
    and assumes that commas (',') are used as decimal separators, based on
    the provided image.

    Args:
        filepath (str): The path to the data file.
        delimiter (str, optional): The column delimiter (e.g., ';', ' '). 
                                   If None, the function will try to 
                                   auto-detect it.

    Returns:
        list: A list of (x, y) float tuples.
              Returns an empty list if the file cannot be read or is empty.
    """
    data_points = []
    detected_delimiter = delimiter
    
    # --- 1. Delimiter Sniffing (if not provided) ---
    if detected_delimiter is None:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                # Read the first non-empty line to guess
                first_line = ""
                for line in f:
                    first_line = line.strip()
                    if first_line:
                        break
                
                if ';' in first_line:
                    detected_delimiter = ';'
                elif ' ' in first_line:
                    # Check if it's likely a space delimiter
                    parts = re.split(r'\s+', first_line)
                    if len(parts) == 2:
                        try:
                            # Try to parse to see if it makes sense
                            float(parts[0].replace(',', '.'))
                            float(parts[1].replace(',', '.'))
                            detected_delimiter = ' '
                        except (ValueError, IndexError):
                             # Not a valid 2-column space-delimited float line
                             pass
                
                if detected_delimiter is None and ',' in first_line:
                    # Comma is the last guess, as it's ambiguous with decimal
                    detected_delimiter = ','
                
                if detected_delimiter is None:
                    # Final fallback based on your image
                    print("Warning: Could not auto-detect delimiter. Falling back to ';'.")
                    detected_delimiter = ';'
        except Exception as e:
            print(f"Error opening/reading file for sniffing: {e}")
            return [] # Return empty list on error
    
    print(f"Using delimiter: '{detected_delimiter}'")

    # --- 2. File Parsing ---
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line_number, line in enumerate(f, 1):
                line = line.strip()
                if not line or line.startswith('#'):
                    continue # Skip empty lines or comment lines

                # Split the line by the detected delimiter
                if detected_delimiter == ' ':
                    # Use regex split for spaces to handle multiple spaces
                    parts = re.split(r'\s+', line)
                else:
                    parts = line.split(detected_delimiter)

                # Ensure we have exactly two columns
                if len(parts) == 2:
                    x_str, y_str = parts
                    
                    try:
                        # KEY STEP: Replace comma with dot for float conversion
                        x_val = float(x_str.strip().replace(',', '.'))
                        y_val = float(y_str.strip().replace(',', '.'))
                        data_points.append((x_val, y_val))
                    except ValueError as e:
                        # Warn if conversion to float fails
                        print(f"Warning: Could not parse numbers on line {line_number}: '{line}'. Error: {e}")
                else:
                    # Warn if the line doesn't have exactly two parts
                    print(f"Warning: Skipping malformed line {line_number}: '{line}'. Expected 2 columns, found {len(parts)}")
    
    except FileNotFoundError:
        print(f"Error: File not found at '{filepath}'")
        return []
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return []

    return data_points

In [49]:
print("--- Example: Reading '19data.csv' ---")
    
# You would replace '19data.csv' with the path to your actual file
file_to_read = '20251GKtest11.csv' 
    
# Check if the file exists before trying to read it
if os.path.exists(file_to_read):
    # Call the function to parse the file.
    # It will try to auto-detect the delimiter.
    all_points = parse_xy_data(file_to_read)
        
    if all_points:
        print(f"Successfully parsed {len(all_points)} data points:")
        df_input = pd.DataFrame(all_points, columns=['x', 'y'])
        print(df_input.to_string(index=False))
    else:
        print(f"Could not parse any data points from '{file_to_read}'.")
        print("Please check the file format and warnings above.")
            
else:
    print(f"Error: The file '{file_to_read}' was not found.")
    print("Please create this file or change 'file_to_read' variable")
    print("to point to your existing data file.")

--- Example: Reading '19data.csv' ---
Using delimiter: ';'
Successfully parsed 104 data points:
     x       y
 1.000 -5.1991
 1.115 -4.3239
 1.230 -3.7284
 1.345 -3.2661
 1.460 -2.9082
 1.575 -2.6225
 1.690 -2.3954
 1.805 -2.2098
 1.920 -2.0282
 2.035 -1.8916
 2.150 -1.7967
 2.265 -1.6635
 2.380 -1.5819
 2.495 -1.5127
 2.610 -1.3977
 2.725 -1.3814
 2.840 -1.2792
 2.955 -1.2786
 3.070 -1.2207
 3.185 -1.1791
 3.300 -1.0930
 3.415 -1.0193
 3.530 -1.0264
 3.645 -0.9742
 3.760 -0.9384
 3.875 -0.8979
 3.990 -0.8110
 4.105 -0.8056
 4.220 -0.7548
 4.335 -0.7416
 4.450 -0.7049
 4.565 -0.6784
 4.680 -0.6445
 4.795 -0.6134
 4.910 -0.5789
 5.025 -0.5578
 5.140 -0.5278
 5.255 -0.4917
 5.370 -0.5137
 5.485 -0.4717
 5.600 -0.4697
 5.715 -0.4589
 5.830 -0.4215
 5.945 -0.4250
 6.060 -0.3984
 6.175 -0.3300
 6.290 -0.3724
 6.405 -0.3815
 6.520 -0.3639
 6.635 -0.3482
 6.750 -0.3021
 6.865 -0.2574
 6.980 -0.3142
 7.095 -0.2815
 7.210 -0.2804
 7.325 -0.2677
 7.440 -0.2675
 7.555 -0.2496
 7.670 -0.2640
 7.7

In [50]:
def get_points_in_range(points: List[Tuple[float, float]], 
                        x_start: float, 
                        x_end: float) -> List[Tuple[float, float]]:
    """
    Filters a list of (x, y) tuples to return only those within a 
    specified x-range [x_start, x_end].
    """
    subset_points = [
        (x, y) for (x, y) in points 
        if x_start <= x <= x_end
    ]
    return subset_points


In [51]:
# Select points for the second interval [9.7, 10.0]
# This interval is suitable for Newton's FORWARD method

x_start_demo = 3.760
x_end_demo = 4.450
    
monotonic_points_fwd = get_points_in_range(all_points, x_start_demo, x_end_demo)
    
print(f"Subset of points from x={x_start_demo} to x={x_end_demo}:")
df_subset = pd.DataFrame(monotonic_points_fwd, columns=['x (swap)', 'y (swap)'])
print(df_subset.to_string(index=False))

Subset of points from x=3.76 to x=4.45:
 x (swap)  y (swap)
    3.760   -0.9384
    3.875   -0.8979
    3.990   -0.8110
    4.105   -0.8056
    4.220   -0.7548
    4.335   -0.7416
    4.450   -0.7049


In [52]:
def EvenDifference(points):
    """
    Calculates the finite difference table for evenly spaced nodes.
    
    Args:
        points (list of tuples): A list of (x, y) data points, sorted by x.

    Returns:
        pandas.DataFrame: The full difference table.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # Format the output DataFrame
    data = {
        'x_i': x_values,
        'y_i': y_values
    }
    for j in range(1, n):
        # Pad with NaN to maintain table shape
        col_name = f'Order {j}'
        col_data = np.full(n, np.nan)
        col_data[:(n-j)] = diff_calc_table[:(n-j), j]
        data[col_name] = col_data
    
    df = pd.DataFrame(data)
    return df


# Stirling


## Algorithm

0. Conditions
* Used for **evenly spaced nodes**: $x_k = x_0 + kh$, $k \in \mathbb{Z}$.
* Requires an **odd number of nodes** (which means an even degree $n$).
* The formula originates from the **central node** $x_0$.
* It is the average of the Gauss I and Gauss II formulas.

1. Finite Difference Table

Given $n+1$ nodes (where $n$ is even), with $x_0$ as the central node.
A standard finite difference table is computed.

*(This is performed by the `EvenDifference(points)` function, which can be found in `Week6-Interpolation/pp2_Newton/pp2b_NewtonFixedGap.ipynb`)*

2. Coefficient Selection

The coefficients $C_i$ (from the slide's formula) are selected from the difference table symmetrically around $y_0$:
* $C_0 = y_0$
* $C_1 = \frac{\Delta y_{-1} + \Delta y_0}{2}$
* $C_2 = \Delta^2 y_{-1}$
* $C_3 = \frac{\Delta^3 y_{-2} + \Delta^3 y_{-1}}{2}$
* $C_4 = \Delta^4 y_{-2}$
* ...
* **General form:**
    * $C_{2k} = \Delta^{2k} y_{-k}$
    * $C_{2k-1} = \frac{\Delta^{2k-1} y_{-k} + \Delta^{2k-1} y_{-(k-1)}}{2}$

3. Polynomial Construction in $t$

The polynomial is constructed in the variable $t = \frac{x - x_0}{h}$.
The formula from the slide is:
$P(t) = C_0 + C_1 \frac{t}{1!} + C_2 \frac{t^2}{2!} + C_3 \frac{t(t^2-1^2)}{3!} + C_4 \frac{t^2(t^2-1^2)}{4!} + C_5 \frac{t(t^2-1^2)(t^2-2^2)}{5!} + \dots$

This can be written as $P(t) = \sum_{i=0}^{n} D_i B_i(t)$

1.  **Calculate Main Coefficients $D_i$:**
    $D_i = \frac{C_i}{i!}$

2.  **Calculate Basis Polynomials $B_i(t)$:**
    * $B_0(t) = 1$
    * $B_1(t) = t$
    * $B_2(t) = t^2$
    * $B_3(t) = t(t^2 - 1^2)$
    * $B_4(t) = t^2(t^2 - 1^2)$
    * $B_5(t) = t(t^2 - 1^2)(t^2 - 2^2)$
    * ...
    * **Recursive form:**
        * $B_{2k}(t) = B_{2k-1}(t) \cdot t$
        * $B_{2k+1}(t) = B_{2k-1}(t) \cdot (t^2 - k^2)$

3.  **Compute Total Polynomial $P(t)$:**
    $P(t) = \sum_{i=0}^{n} N_i(t) = \sum_{i=0}^{n} D_i B_i(t)$
    * The coefficients of each $N_i(t)$ are summed by degree to find the final coefficients of $P(t) = a_0 + a_1 t + \dots + a_n t^n$.

4.  **Output:** Return the intermediate steps table and the final coefficient list $a_k$ for $P(t)$.

In [53]:
def stirling_interpolation(points, x0_index):
    """
    Constructs the Stirling interpolation polynomial in the variable t = (x - x0) / h.
    
    Args:
        points (list of tuples): List of (x, y) data points. Must be evenly spaced
                                 and have an odd number of points.
        x0_index (int): The index of the central node (x0).

    Returns:
        step_pd (pd.DataFrame): DataFrame showing the intermediate steps.
        coeff_pd (pd.DataFrame): DataFrame of the final polynomial coefficients for P(t).
    """
    
    n = len(points) - 1
    if (n + 1) % 2 == 0:
        raise ValueError("Stirling's formula requires an odd number of nodes (even degree n).")
    
    # --- 1. Generate Difference Table ---
    diff_table_df = EvenDifference(points)

    # --- 2. Select Coefficients (Stirling Path) ---
    C_coeffs = []
    for i in range(n + 1):
        order_col = f'Order {i}' if i > 0 else 'y_i'
        
        if i == 0:
            # C_0 = y_0
            C_coeffs.append(diff_table_df[order_col].iloc[x0_index])
        elif i % 2 == 1: # Odd order: C_{2k-1} = (d_{2k-1} y_{-k} + d_{2k-1} y_{-(k-1)}) / 2
            k = (i + 1) // 2
            c1 = diff_table_df[order_col].iloc[x0_index - k]
            c2 = diff_table_df[order_col].iloc[x0_index - k + 1]
            C_coeffs.append((c1 + c2) / 2.0)
        else: # Even order: C_{2k} = d_{2k} y_{-k}
            k = i // 2
            C_coeffs.append(diff_table_df[order_col].iloc[x0_index - k])

    # --- 3. Build Polynomial P(t) ---
    steps_data = []
    N_coeffs_total = np.zeros(n + 1, dtype=float)
    
    # B_i(t) basis polynomials
    B_coeffs_list = []
    
    # B_0(t) = 1
    B_0 = np.array([1.0])
    B_coeffs_list.append(B_0)
    
    # B_1(t) = t
    B_1 = np.array([0.0, 1.0]) # [const, t]
    B_coeffs_list.append(B_1)

    # *** CORRECTED LOGIC FOR B_i(t) ***
    for i in range(2, n + 1):
        if i % 2 == 0: # Even: B_{2k} = t * B_{2k-1}
            # B_{2k-1} is the previous basis poly, at index i-1
            B_i = np.convolve(B_coeffs_list[i-1], [0, 1])
        else: # Odd: B_{2k+1} = (t^2 - k^2) * B_{2k-1}
            k = (i - 1) // 2
            # B_{2k-1} is the basis poly at index i-2
            B_i = np.convolve(B_coeffs_list[i-2], [-k**2, 0, 1.0])
        
        B_coeffs_list.append(B_i)

    # Calculate final polynomial
    for i in range(n + 1):
        D_i = C_coeffs[i] / math.factorial(i)
        B_i = B_coeffs_list[i]
        
        # N_i(t) = D_i * B_i(t)
        Ni_coeffs = D_i * B_i
        
        # Add to total polynomial (pad with zeros)
        N_coeffs_total[:len(Ni_coeffs)] += Ni_coeffs

        # Store intermediate steps for printing
        steps_data.append({
            'i': i,
            'Difference Coeff (C_i)': C_coeffs[i],
            'D_i = C_i / i!': D_i,
            'B_i(t) Coeffs (low->high)': B_i.tolist(),
            'N_i(t) Coeffs (low->high)': Ni_coeffs.tolist()
        })

    step_pd = pd.DataFrame(steps_data)
    coeff_pd = pd.DataFrame({
        'Degree (t)': np.arange(n + 1),
        'Coeff': N_coeffs_total
    })

    return step_pd, coeff_pd

## Result

In [54]:
# 1. Define the data points from the image
points = monotonic_points_fwd
# 2. Set the central node index
# 9 points (indices 0-8), the center is index 4
x0_index = 3
x0_val = points[x0_index][0]
h = points[1][0] - points[0][0]

In [55]:
print("--- Generated Finite Difference Table ---")
df = EvenDifference(points)

df.style

--- Generated Finite Difference Table ---


,x_i,y_i,Order 1,Order 2,Order 3,Order 4,Order 5,Order 6
0,3.760000,-0.938400,0.040500,0.046400,-0.127900,0.254800,-0.464700,0.818700
1,3.875000,-0.897900,0.086900,-0.081500,0.126900,-0.209900,0.354000,nan
2,3.990000,-0.811000,0.005400,0.045400,-0.083000,0.144100,nan,nan
3,4.105000,-0.805600,0.050800,-0.037600,0.061100,nan,nan,nan
4,4.220000,-0.754800,0.013200,0.023500,nan,nan,nan,nan
5,4.335000,-0.741600,0.036700,nan,nan,nan,nan,nan
6,4.450000,-0.704900,nan,nan,nan,nan,nan,nan


In [56]:
# 3. Calculate the Stirling polynomial
step_df, final_coeff_df = stirling_interpolation(points, x0_index)

print("\n--- Polynomial Construction Steps (Stirling) ---")
step_df.style


--- Polynomial Construction Steps (Stirling) ---


,i,Difference Coeff (C_i),D_i = C_i / i!,B_i(t) Coeffs (low->high),N_i(t) Coeffs (low->high)
0,0,-0.805600,-0.805600,[1.0],[-0.8056]
1,1,0.028100,0.028100,"[0.0, 1.0]","[0.0, 0.028100000000000014]"
2,2,0.045400,0.022700,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.022699999999999942]"
3,3,0.021950,0.003658,"[0.0, -1.0, 0.0, 1.0]","[0.0, -0.003658333333333328, 0.0, 0.003658333333333328]"
4,4,-0.209900,-0.008746,"[0.0, 0.0, -1.0, 0.0, 1.0]","[-0.0, -0.0, 0.008745833333333319, -0.0, -0.008745833333333319]"
5,5,-0.055350,-0.000461,"[0.0, 4.0, 0.0, -5.0, 0.0, 1.0]","[-0.0, -0.0018449999999999966, -0.0, 0.002306249999999996, -0.0, -0.00046124999999999915]"
6,6,0.818700,0.001137,"[0.0, 0.0, 4.0, 0.0, -5.0, 0.0, 1.0]","[0.0, 0.0, 0.004548333333333327, 0.0, -0.005685416666666659, 0.0, 0.0011370833333333317]"


In [57]:
final_coeff_df.style

,Degree (t),Coeff
0,0,-0.805600
1,1,0.022597
2,2,0.035994
3,3,0.005965
4,4,-0.014431
5,5,-0.000461
6,6,0.001137


## Further Testing

In [58]:
#Horner Test
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)

        Actual differentation P'(k) (x) = P'(k) (c) / h^k
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

def all_derivatives(a, c, gap):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []
    actual_dev_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        actual_dev_list.append(b0 * math.factorial(i) / gap**i)
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list
    df.loc["p^(i)(x)"] = [None] + actual_dev_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

In [60]:
coeff_list = final_coeff_df['Coeff'].tolist()

x_val = 4.15
t_val = (x_val - x0_val) / h

print(f"x0_val: {x0_val}; gap: {h}")

df2 = all_derivatives(coeff_list, t_val, h)
df2.style

x0_val: 4.105; gap: 0.11500000000000021


,a_i,i=0,i=1,i=2,i=3,i=4,i=5,i=6
c,0.391304,nan,nan,nan,nan,nan,nan,nan
0,-0.805600,-0.791228,0.050056,0.029861,-0.015967,-0.012722,0.002208,0.001137
1,0.022597,0.036730,0.034056,-0.010719,-0.013412,0.001763,0.001137,nan
2,0.035994,0.036117,-0.005269,-0.013928,0.001319,0.001137,nan,nan
3,0.005965,0.000315,-0.014270,0.000874,0.001137,nan,nan,nan
4,-0.014431,-0.014438,0.000429,0.001137,nan,nan,nan,nan
5,-0.000461,-0.000016,0.001137,nan,nan,nan,nan,nan
6,0.001137,0.001137,nan,nan,nan,nan,nan,nan
b_0,nan,-0.791228,0.050056,0.029861,-0.015967,-0.012722,0.002208,0.001137
p^(i)(c),nan,-0.791228,0.050056,0.059723,-0.095803,-0.305329,0.265011,0.818700
